In [25]:
import pandas as pd
import numpy as np  
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50
pd.options.mode.chained_assignment = None

# from itables import init_notebook_mode
# init_notebook_mode(all_interactive=True)


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("PAT_FLAGS.csv")
print(df.shape)

(551127, 722)


In [26]:
# Dictionary of clinical concepts
clinDict = {"A1": "AcuteStressReaction",
    "A2": "AdjustmentDisorder",
    "A3":"Alcohol",
    "A4":"AntisocialBehavior",
    "A5":"AttentionDeficit",
    "A6":"BipolarI",
    "A7":"BipolarII",
    "A8":"Cannabis",
    "A9":"Cocaine",
    "A10":"ConductDisorder",
    "A11":"ConversionDisorder",
    "A12":"DissociativeDisorder",
    "A13":"EatingDisorder",
    "A14":"ExplosiveDisorder",
    "A15":"FactitiousDisorder",
    "A16":"GeneralizedAnxiety",
    "A17":"TobaccoUse",
    "A18":"HomicidalIdeation",
    "A19":"ImpulseControl",
    "A20":"MajorDepressive",
    "A21":"ObsessiveCompulsive",
    "A22":"Opioid",
    "A23":"FLAG_Other",
    "A24":"OtherAnxiety",
    "A25":"OtherBipolar",
    "A26":"OtherDepressive",
    "A27":"OtherManic",
    "A28":"OtherMood",
    "A29":"OtherPsychosis",
    "A30":"OtherStress",
    "A31":"PainDisorder",
    "A32":"PanicDisorder",
    "A33":"AdultAbuse",
    "A34":"Combat",
    "A35":"Trauma",
    "A36":"PersonalityDisorder",
    "A37":"PhobicDisorder",
    "A38":"PTSD",
    "A39":"Schizophrenia",
    "A40":"SelfHarm",
    "A41":"SomatoformDisorder",
    "A42":"OtherSubstance",
    "A43":"TobaccoSubstance",
    "A44":"SuicideAttempt",
    "A45":"SuicideIdeation"}

In [27]:
# Create df with desired variables
df2 = df.filter(regex='_FLAG|_DATE_MIN_PRE|_DATE_MIN_POST|_DATE_MIN_DOD|_DATE_MIN_VA')
print(df2.shape)

(551127, 225)


In [28]:
dfs = [] # To store and combine dfs

for k in clinDict:
    
    def count_table(df, k):
        flag = "_FLAG"
        preTBI = "_DATE_MIN_PRE"
        postTBI = "_DATE_MIN_POST"
        dod = "_DATE_MIN_DOD"
        va = "_DATE_MIN_VA"

        kflag = k + flag
        kpre = k + preTBI
        kpost = k + postTBI
        kdod = k + dod
        kva = k + va

        df = df[[kflag, kpre, kpost, kdod, kva]]

        df = df.rename(columns={kflag: "Flag", kpre: "Pre-TBI",
                               kpost: 'Post-TBI',
                               kdod: "DOD",
                               kva: "VA"})
        return df

# Function to generate table of counts and percent. 
    def transform_dataframe(df1):
        column_names = []
        n_values = []
        percentage_values = []

        for column in df1.columns:
            column_names.append(column)
            valid_cells = df1[column][(df1[column] != 0) & (~df1[column].isnull())].count()
            n_values.append(valid_cells)
            percentage_values.append(valid_cells / len(df1[column]) * 100)

        data = {
            "Variable": column_names,
            "n": n_values,
            "%": percentage_values
        }

        df2 = pd.DataFrame(data)
        df2.insert(0, "Clinical Concept", k)
        # df2 = df2.style.hide_index().set_caption(clinDict[k])
#         df2 = df2.style.hide_index()
        
        return df2
    
    df1 = count_table(df, k)
    df2 = transform_dataframe(df1)
    # display(df2) # prints all 45 tables 
    
    dfs.append(df2)

In [42]:
# Combine dfs
mergedDf = pd.concat(dfs, axis=0, ignore_index=False)
# mergedDf= mergedDf.set_index(['Clinical Concept']) # We can set concept as index if you want but that would impact the search drop down.
mergedDf.shape
# display(mergedDf)

(225, 4)

In [30]:
import ipywidgets as widgets

In [43]:
my_list = list(clinDict.keys())
my_list.append("All")
# print([my_list])

In [44]:
# def view1(ClinicalDx):
#     display(mergedDf[mergedDf['Clinical Concept']==ClinicalDx])

# clinSelect = widgets.Select(options=my_list)
# widgets.interactive(view1, ClinicalDx=clinSelect) # We can replace Clinical Concept with actual Dx too if desired.

# Adding ALL option. Delete the above if we don't want all option.
def view1(ClinicalDx):
    if ClinicalDx == 'All':
        display(mergedDf)
    else:
        display(mergedDf[mergedDf['Clinical Concept']==ClinicalDx])

clinSelect = widgets.Select(options=my_list)
widgets.interactive(view1, ClinicalDx=clinSelect)

interactive(children=(Select(description='ClinicalDx', options=('A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8'…

In [ ]:
# ADD another table with greatest counts for each dx etc. 
# ADD dropdown, exact match search, maybe clinical concept, title/aes. 